# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [3]:
%pip install duckdb duckdb-engine jupysql leafmap
import duckdb
import leafmap

In [4]:
%load_ext sql
# Now Configure sql magic # to automatically return query results as a Pandas DF
%config SqlMagic.autopandas = True
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [10]:
# Connect to the DuckDB database
%sql duckdb:///nyc_data.db

In [11]:
#installing and loading httpfs extension, which allows reading/writing files over HTTP
%%sql
INSTALL httpfs;
LOAD httpfs;

,Success


Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [9]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

nyc_data.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.zip'

In [12]:
%%sql

CREATE TABLE nyc_census_blocks AS SELECT * FROM "nyc_census_blocks.shp";
CREATE TABLE nyc_homicides AS SELECT * FROM  "nyc_homicides.shp";
CREATE TABLE nyc_neighborhoods AS SELECT * FROM "nyc_neighborhoods.shp";
CREATE TABLE nyc_streets AS SELECT * FROM "nyc_streets.shp";
CREATE TABLE nyc_subway_stations AS SELECT * FROM "nyc_subway_stations.shp";

,Success


In [23]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';


,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [35]:
%%sql

SELECT * FROM nyc_subway_stations LIMIT 3

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [29]:
%%sql
SELECT * FROM nyc_neighborhoods;


,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[2, 4, 0, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198, ..."
1,Manhattan,East Village,"[2, 4, 0, 0, 0, 0, 0, 0, 35, 215, 14, 73, 139,..."
2,Manhattan,West Village,"[2, 4, 0, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212,..."
3,The Bronx,Throggs Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 128, 232, 17, 73, 174..."
4,The Bronx,Wakefield-Williamsbridge,"[2, 4, 0, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17, 2..."
...,...,...,...
124,Brooklyn,Red Hook,"[5, 4, 0, 0, 0, 0, 0, 0, 18, 0, 14, 73, 149, 8..."
125,Queens,Douglastown-Little Neck,"[2, 4, 0, 0, 0, 0, 0, 0, 251, 165, 19, 73, 76,..."
126,Queens,Whitestone,"[5, 4, 0, 0, 0, 0, 0, 0, 17, 75, 18, 73, 86, 2..."
127,Queens,Steinway,"[5, 4, 0, 0, 0, 0, 0, 0, 124, 87, 16, 73, 87, ..."


In [33]:
%%sql

SELECT nyc_s."NAME", nyc_s."ROUTES" FROM nyc_subway_stations As nyc_s
JOIN nyc_neighborhoods AS nyc_n
ON ST_WITHIN(nyc_s.geom, nyc_n.geom)
WHERE nyc_n."NAME" = 'Little Italy'

,NAME,ROUTES
0,Spring St,6


In [32]:
%%sql

SELECT nyc_s."NAME", nyc_s."ROUTES" FROM nyc_subway_stations As nyc_s
JOIN nyc_neighborhoods AS nyc_n
ON ST_Contains(nyc_n.geom, nyc_s.geom)
WHERE nyc_n."NAME" = 'Little Italy'

,NAME,ROUTES
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [40]:
%%sql

SELECT DISTINCT nyc_n."NAME", nyc_n."BORONAME" FROM nyc_neighborhoods AS nyc_n
JOIN nyc_subway_stations As nyc_s ON ST_WITHIN(nyc_s.geom, nyc_n.geom)
WHERE nyc_s."ROUTES" LIKE '%6%';


,NAME,BORONAME
0,Chinatown,Manhattan
1,Greenwich Village,Manhattan
2,Gramercy,Manhattan
3,Murray Hill,Manhattan
4,Midtown,Manhattan
5,Yorkville,Manhattan
6,Soundview,The Bronx
7,Parkchester,The Bronx
8,Financial District,Manhattan
9,Little Italy,Manhattan


In [41]:
%%sql

SELECT DISTINCT nyc_n."NAME" , nyc_n."BORONAME" FROM nyc_neighborhoods AS nyc_n
JOIN nyc_subway_stations As nyc_s ON ST_WITHIN(nyc_s.geom, nyc_n.geom)
WHERE strpos(nyc_s."ROUTES",'6') > 0;

,NAME,BORONAME
0,Financial District,Manhattan
1,Little Italy,Manhattan
2,Upper East Side,Manhattan
3,East Harlem,Manhattan
4,Mott Haven,The Bronx
5,Hunts Point,The Bronx
6,South Bronx,The Bronx
7,Chinatown,Manhattan
8,Greenwich Village,Manhattan
9,Gramercy,Manhattan


In [39]:
%%sql

SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0;

,NAME,BORONAME
0,Financial District,Manhattan
1,Little Italy,Manhattan
2,Upper East Side,Manhattan
3,East Harlem,Manhattan
4,Mott Haven,The Bronx
5,Hunts Point,The Bronx
6,South Bronx,The Bronx
7,Chinatown,Manhattan
8,Greenwich Village,Manhattan
9,Gramercy,Manhattan


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [42]:
%%sql

SELECT * FROM nyc_census_blocks LIMIT 3;

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."


In [44]:
%%sql

SELECT Sum(nyc_c."POPN_TOTAL") AS evac_pop
FROM nyc_census_blocks AS nyc_c
JOIN nyc_neighborhoods AS nyc_n
ON ST_Intersects(nyc_c.geom, nyc_n.geom)
WHERE nyc_n."NAME" = 'Battery Park'

,evac_pop
0,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [46]:
%%sql

SELECT nyc_n."NAME" , SUM(nyc_c."POPN_TOTAL")/(ST_Area(nyc_n.geom) / 1000000.0) AS popn_per_sqkm
FROM  nyc_census_blocks AS nyc_c
JOIN nyc_neighborhoods AS nyc_n
ON ST_Intersects(nyc_c.geom, nyc_n.geom)
GROUP BY nyc_n.name, nyc_n.geom
ORDER BY popn_per_sqkm DESC LIMIT 2;

,NAME,popn_per_sqkm
0,North Sutton Area,68435.132838
1,East Village,50404.483413


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [47]:
%pip install duckdb duckdb-engine jupysql

In [48]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [49]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [52]:
%%sql

DROP TABLE IF EXISTS AIS;

CREATE TABLE AIS AS SELECT * FROM 'https://storage.googleapis.com/qm2/casa0025_ships.csv';


ALTER TABLE AIS ADD COLUMN geom GEOMETRY;

UPDATE AIS
SET geom = ST_SetSRID(ST_MakePoint(lon, lat), 4326);



RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Column with name geom already exists!
[SQL: ALTER TABLE AIS ADD COLUMN geom GEOMETRY;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.